In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [34]:
class Player:
    def __init__(self, player, post, age, market_value, nat, joined, league, fee):
        self.PlayerName = player
        self.Post = post
        self.Age = age
        self.MarketValue = market_value
        self.Nationality= nat
        self.TeamJoined = joined
        self.League = league
        self.Fee = fee


In [35]:
def get_trans(page):
    url=f"https://www.transfermarkt.com/statistik/saisontransfers?ajax=yw0&page={page}"
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
                  "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1",
                  "Connection":"close", "Upgrade-Insecure-Requests":"100"}
    page= requests.get(url, headers= headers)
    soup= BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', class_='items')
    rows = table.find_all('tr', class_=['odd', 'even'])
    players = []
    for row in rows:
        name=row.find_all('td')[1].find('a').text
        post=row.find_all('tr')[1].find('td').text
        age=row.find_all('td')[5].text
        market_value=row.find_all('td')[6].text.replace('€', '').strip()
        nat=row.find_all('td')[7].find('img')['alt']
        joined=row.find_all('td')[8].find_all('a')[0]['title']
        legue=row.find_all('td')[8].find_all('a')[2]['title']
        fee=row.find_all('td')[12].text.replace('€', '').strip()
        player = Player(name, post, age, market_value, nat, joined, legue, fee)
        players.append(player)
    players = [vars(player) for player in players]
    df = pd.DataFrame(players)
    return df

In [36]:
df=get_trans(2)



In [37]:
dff=df.copy()


In [38]:
def convertir_valeur(valeur):
    if isinstance(valeur, str):
        if 'k' in valeur and len(valeur) < 10:
            return float(valeur.replace('k', '')) / 1000
        elif 'm' in valeur and len(valeur) < 10:
            return float(valeur.replace('m', ''))
        elif 'Loan fee' in valeur:
            value = valeur.split(":")[1]
            if 'k' in value:
                return float(value.replace('k', '')) / 1000
            elif 'm' in value:
                return float(value.replace('m', ''))
        else:
            try:
                return float(valeur)
            except ValueError:
                # En cas d'échec, retourner NaN
                return valeur
    else:
        return valeur

# Appliquer la fonction à la colonne market_value
dff['MarketValue'] = dff['MarketValue'].apply(convertir_valeur)
dff['Fee'] = dff['Fee'].apply(convertir_valeur)


In [39]:
dff

,PlayerName,Post,Age,MarketValue,Nationality,TeamJoined,League,Fee
0,Rúben Neves,Defensive Midfield,26,40.0,Portugal,Al-Hilal SFC,Saudi Pro League,55.0
1,Aleksandar Mitrovic,Centre-Forward,28,28.0,Serbia,Al-Hilal SFC,Saudi Pro League,52.6
2,Matheus Cunha,Centre-Forward,24,25.0,Brazil,Wolverhampton Wanderers,Premier League,50.0
3,Sergej Milinković-Savić,Central Midfield,28,50.0,Serbia,Al-Hilal SFC,Saudi Pro League,40.0
4,Dejan Kulusevski,Right Winger,23,50.0,Sweden,Tottenham Hotspur,Premier League,30.0
5,João Félix,Left Winger,23,50.0,Portugal,FC Barcelona,LaLiga,loan transfer
6,João Cancelo,Right-Back,29,50.0,Portugal,FC Barcelona,LaLiga,loan transfer
7,Milan Skriniar,Centre-Back,28,50.0,Slovakia,Paris Saint-Germain,Ligue 1,free transfer
8,Gonçalo Ramos,Centre-Forward,22,50.0,Portugal,Paris Saint-Germain,Ligue 1,loan transfer
9,Cole Palmer,Attacking Midfield,21,18.0,England,Chelsea FC,Premier League,47.0


In [115]:
dff.to_csv('players.csv', index=False)